# Loan Borrowers Reliability Analysis

**Introduction**

The customer for this project is the bank’s credit department. The task is to determine whether a client’s marital status and number of children affect their ability to repay a loan on time. The dataset provided by the bank contains statistics on customer creditworthiness.

The findings from this analysis will later be used to improve a credit scoring model - a specialized system that assesses a potential borrower’s ability to repay a loan to the bank.

**Data Description:**
- `children` - number of children in the family,
- `days_employed` - total work experience (in days),
- `dob_years` - client's age (in years),
- `education` - client's educational level,
- `education_id` - education level identifier,
- `family_status` - marital status,
- `family_status_id` - marital status identifier,
- `gender` - client's gender,
- `income_type` - employment type,
- `debt` - was there any debt on repayment of loans,
- `total_income` - monthly income,
- `purpose` - purpose of the loan.

## 1. Examining data

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/data.csv')

In [ ]:
data.head(20)

In [ ]:
data.info()

The dataset was loaded using Pandas and ran `.head()` and `.info()` to get a first look at the data. This initial inspection helped understand the table structure, identify columns with missing values (such as `days_employed` and `total_income`), and verify data types.

## 2. Data preprocessing

### 2.1 Gaps removal

In [ ]:
data.isna().sum() # number of missing values ​​for each column

Missing values were identified in both the `days_employed` and `total_income` columns. `days_employed` will be handled in the next step.

For `total_income`, since earnings are strongly influenced by employment status, missing values were filled with the median income calculated for each specific `income_type` (e.g., filling in missing values for "employee" with the median income of all employees).

In [ ]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == type, 'total_income'].median() 

### 2.2 Handling abnormal values

Anomalies were identified in the `days_employed` column: negative values, which are likely due to technical errors. To fix this, it was decided to replace all negative values with their absolute values using the `abs()` method.

In [ ]:
data['days_empoyed'] = data['days_employed'].abs() # replacing all negative values ​​with positive ones

Calculating median `days_employed` for each `income_type`

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

It was noticed that the "unemployed" and "retirees" groups have abnormally high values. Since fixing these specific data errors is complex and this column is not critical for this specific investigation, it was decided to leave them as is.

In [ ]:
data['children'].unique() # list of unique values ​​of the children column

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)] # deleting rows with abnormal values ​​(-1 and 20)

In [ ]:
data['children'].unique()

### 2.3 Gaps removal (part 2)

Next, the blanks in the `days_employed` column will be filled with the median values ​​for each employment type `income_type`

In [ ]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == type), 'days_employed'].median()

In [ ]:
data.isna().sum()

### 2.4 Data Types Conversion

In [ ]:
data['total_income'] = data['total_income'].astype(int) # replacing float64 data type with int one

### 2.5 Duplicate handling

The `education` column was standardized by converting all values to lowercase to eliminate implicit duplicates caused by inconsistent capitalization.

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### 2.6 Categorization of data

Here a `total_income_category` column was created by binning the continuous income data into five specific ranges (labeled A through E). This was implemented using a custom function `categorize_income()` and the `.apply()` method.

- 0-30000 - `'E'`;
- 30001-50000 - `'D'`;
- 50001-200000 - `'C'`;
- 200001-1000000 - `'B'`;
- 1000001 and more - `'A'`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [ ]:
data['purpose'].unique()

Various raw loan descriptions here were consolidated into four main categories:

- `'deals with a car'`,
- `'deals with real estate'`,
- `'conducting a wedding'`,
- `'getting an education'`.

It's a function `categorize_purpose()` to detect key substrings within the `purpose` column and assign the standardized category.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'deals with a car'
        elif 'жил' in row or 'недвиж' in row:
            return 'deals with real estate'
        elif 'свад' in row:
            return 'conducting a wedding'
        elif 'образов' in row:
            return 'getting an education'
    except:
        return 'no category' 

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3. EDA and questions

To streamline the analysis for questions 3.1–3.4, a reusable function `create_pivot()` was created. This function generates pivot tables, simplifying the data aggregation required to answer each question.

In [ ]:
def create_pivot(column):
    new_table = pd.pivot_table(data,
                index=column,
                values=['debt'],
                aggfunc=['count', 'sum', 'mean'])
    new_table.columns = ['Количество заемщиков', 'Сумма просрочек', 'Доля просрочек']
    return new_table

### 3.1 Is there a correlation between the number of children and timely loan repayment?

In [ ]:
create_pivot('children')

Based on the data obtained in the pivot table above, which shows the relationship between the number of children and on-time loan repayment, clients (borrowers) with 4 and 5 children account for only 0.24% of the dataset. Therefore, they were excluded from this analysis.

**Conclusion**: The remaining borrowers can be divided into three groups:

- Group 1 (0 children): This group has the lowest default rate. They are the most reliable payers and make up the majority of the dataset (~66%).
- Group 2 (1–2 children): This group has the highest default rate, indicating a higher risk of missed payments. They represent ~32% of the dataset.
- Group 3 (3 children): Their default rate falls in the middle (between Groups 1 and 2). They represent ~2% of the dataset.

**Interpretation**: The analysis indicates that borrowers without children are more likely to repay loans on time. This is likely because they can plan their budgets with more certainty. Conversely, borrowers with children may face unforeseen child-related expenses that disrupt their financial planning, leading to a higher frequency of missed payments.

### 3.2 Is there a correlation between marital status and timely loan repayment?

In [ ]:
create_pivot('family_status')

**Conclusion**: Based on the pivot table analysis, borrowers can be categorized into three distinct groups:

- Group 1 (Widowed or Divorced): This segment represents a small portion of the dataset (~10%), but they demonstrate the highest reliability in repayment.
- Group 2 (Married): This is the largest segment (~57%). Their repayment reliability is average (moderate default rate).
- Group 3 (Civil Partnership or Unmarried): This group makes up ~33% of the data. They have the highest default rate (least reliable).

**Interpretation**: We can hypothesize that Group 1 consists of older, independent individuals with high financial discipline. Group 2 likely benefits from dual incomes and joint budget planning. Group 3 may consist of younger individuals who lack financial discipline or stability regarding credit obligations.

### 3.3 Is there a correlation between income level and timely loan repayment?

To answer this question, I decided to modify the income categorization. The categories proposed in the earlier tasks resulted in a highly uneven distribution of borrowers.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 50000:
            return 'I:0 - 50,000'
        elif 50001 <= income <= 100000:
            return 'H:50,001 - 100,000'
        elif 100001 <= income <= 150000:
            return 'G:100,001 - 150,000'
        elif 150001 <= income <= 200000:
            return 'F:150,001 - 200,000'
        elif 200001 <= income <= 250000:
            return 'E:200,001 - 250,000'
        elif 250001 <= income <= 300000:
            return 'D:250,001 - 300,000'
        elif 300001 <= income <= 400000:
            return 'C:300,001 - 400,000'
        elif 400001 <= income <= 500000:
            return 'B:400,001 - 500,000'
        elif income >= 500001:
            return 'A:Above 500,001'
    except:
        pass

In [ ]:
data['total_income_categoty'] = data['total_income'].apply(categorize_income)

In [ ]:
create_pivot('total_income_category')

**Conclusion**: Based on the analysis of this table, I identified 4 distinct borrower groups based on the relationship between their income and repayment reliability:

- Group 1 (Category B): Represents ~2% of the dataset. These borrowers consistently repay debts on time.
- Group 2 (Categories A, D, and I): Represents ~9% of the dataset. Their default rate is slightly higher, but they generally manage to meet payment deadlines.
- Group 3 (Categories C and E): Represents ~15% of the dataset. Their default rate is higher, and they carry more debt than the previous groups.
- Group 4 (Categories F, G, and H): Represents the majority (~74%) of the dataset. This group has the highest default rate and misses payment deadlines most frequently.

**Interpretation**: The analysis suggests that borrowers in Groups 1 and 2 are the most reliable, but for different reasons. They likely include those with very low income (who avoid taking large loans and pay on time to avoid penalties) and those with very high income (who have the financial cushion to handle payments without stress).

Regarding Groups 3 and 4 (the low-to-middle income segments), we can hypothesize that these borrowers face a difficult debt-to-income ratio. They likely have multiple credit obligations that are difficult to service with their current income levels, leading to a higher frequency of missed payments.

***Insight after project completion***

In this project, there was no need to rework the categorization, as the client had already proposed their own categories. However, it is important to note that the groups are extremely unbalanced: the smallest category is 728 times smaller than the largest. This could affect the results.

A custom categorization could be proposed as additional research. In the future, when categorizing quantitative data, it is best to make the categories roughly equal. To achieve this, there is an option to use the `qcut()` method.

### 3.4 How do different loan purposes affect on-time repayment?

In [ ]:
create_pivot('purpose_category')

**Conclusion**: To simplify the analysis, it is effective to cluster the loan purposes into two main groups:

- Group 1 (Real Estate & Weddings): Representing ~61% of the dataset. These borrowers have fewer debts and generally repay on time.
- Group 2 (Cars & Education): Representing ~39% of the dataset. These borrowers have higher debt levels and a higher frequency of missed payments.

**Interpretation**: We can hypothesize the reasons for these differences:

- For Group 1: These loans often involve couples (joint real estate, weddings paid from a shared budget), or the borrowers are simply more financially prepared for such significant, planned expenses.
- For Group 2: These borrowers face specific financial strains. Students may find it difficult to balance work and study, impacting their income stability. Car owners often face additional unexpected costs (maintenance, repairs, insurance) that reduce their ability to make loan payments on time.

### 3.5 What are the possible reasons for missing values in the source data?

Missing values in this dataset can be attributed to both technical issues and human factors.

1. ***Technical Issues***:

- **Data Migration**: Data might have been lost during a transfer between servers or databases. This likely explains the NaN values in the `days_employed` and `total_income` columns.
- **Calculation Errors**: A technical error in the calculation algorithm likely caused the negative values found in the `days_employed` column.

2. ***Human Factors***:

- **Optional Fields**: Regarding `days_employed`, a bank employee or the borrower may have left the field blank if it was optional, intending to fill it in later, but failing to do so.
- **Privacy Concerns**: regarding `total_income`, borrowers may have simply chosen not to disclose this sensitive financial information.

### 3.6 Explain why median imputation (filling gaps with the median) is the best solution for quantitative variables

Filling quantitative variables (such as `days_employed` and `total_income`) with the median value was the best approach because the mean (average) does not accurately represent data when outliers are present. When certain values stand out significantly from the majority, the mean gives a distorted picture. For example, consider the extreme values in the `total_income` column:

In [ ]:
data['total_income'].max()

In [ ]:
data['total_income'].min()

The difference between these two income figures is massive, and they represent outliers compared to the rest of the dataset. Therefore, using the arithmetic mean for these quantitative variables would be incorrect; it would skew the data and lead to an inaccurate analysis.

## 4. Conclusion

As part of the analysis to build a credit scoring model, I tested four hypotheses. Hypotheses 1 and 2 are the primary factors for the model.

1. *Is there a correlation between the number of children and on-time loan repayment?*

Yes. Clients with no children constitute the largest group and have the lowest default rate. Conversely, borrowers with 1–3 children show the highest rate of missed payments.

2. *Is there a correlation between marital status and on-time loan repayment?*

Yes. The 'widow/widower' and 'divorced' categories have the lowest default rates. The 'civil partnership' and 'unmarried' categories have the highest default rates. The largest group, 'married', shows an average default rate.

3. *Is there a correlation between income level and on-time loan repayment?*

Yes. Borrowers in the '50,000 – 200,000' income range represent the largest group and have the highest default rate. In contrast, the groups with extreme income values (very low <50,000 and very high >500,000) demonstrate lower default rates.

4. *How do different loan purposes affect on-time repayment?*

The analysis showed that 'real estate' transactions are the most common loan purpose. Borrowers taking loans for 'real estate' and 'weddings' have a low default rate. In contrast, loans for 'car operations' and 'education' are associated with higher default rates and debt levels.